<a href="https://colab.research.google.com/github/jlonge4/gen_ai_utils/blob/main/phi4_r1_grpo_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers vllm accelerate

In [1]:
def format_func(reference, query, response):
    prompt = f"""Your job is to evaluate whether a machine learning model has hallucinated or not.
  A hallucination occurs when the response is coherent but factually incorrect or nonsensical
  outputs that are not grounded in the provided context.
  You are given the following information:
      ####INFO####
      [Knowledge]: {reference}
      [User Input]: {query}
      [Model Response]: {response}
      ####END INFO####
      Based on the information provided is the model output a hallucination?
      """
    return prompt

In [2]:
reference = 'The capital of France is Paris. The population is around 2 million.'
query = 'What is the population of the capital of France?'
response = 'London, the capital of France has about 429 million people.'

prompt = format_func(reference, query, response)
print(prompt)

Your job is to evaluate whether a machine learning model has hallucinated or not.
  A hallucination occurs when the response is coherent but factually incorrect or nonsensical
  outputs that are not grounded in the provided context.
  You are given the following information:
      ####INFO####
      [Knowledge]: The capital of France is Paris. The population is around 2 million.
      [User Input]: What is the population of the capital of France?
      [Model Response]: London, the capital of France has about 429 million people.
      ####END INFO####
      Based on the information provided is the model output a hallucination?
      


In [3]:
from vllm import LLM, SamplingParams

# Configure sampling parameters
sampling_params = SamplingParams(
    temperature=0.1,
    top_p=0.1,
    max_tokens=500,
)

# Initialize the LLM
llm = LLM(
    model="Jlonge4/phi4-r1-merge",  # Local path or HuggingFace model ID
    max_num_seqs=1,
    max_model_len=512,
    tensor_parallel_size=1,  # Increase for multi-GPU inference
    gpu_memory_utilization=0.9,
)

INFO 02-15 03:28:42 __init__.py:190] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 02-15 03:28:57 config.py:542] This model supports multiple tasks: {'reward', 'classify', 'embed', 'generate', 'score'}. Defaulting to 'generate'.
INFO 02-15 03:28:57 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='Jlonge4/phi4-r1-merge', speculative_config=None, tokenizer='Jlonge4/phi4-r1-merge', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=512, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=Jlonge4/phi4-r1-merge, num_scheduler_steps=

Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]


INFO 02-15 03:29:10 model_runner.py:1115] Loading model weights took 27.3876 GB
INFO 02-15 03:29:11 worker.py:267] Memory profiling takes 1.00 seconds
INFO 02-15 03:29:11 worker.py:267] the current vLLM instance can use total_gpu_memory (39.56GiB) x gpu_memory_utilization (0.90) = 35.60GiB
INFO 02-15 03:29:11 worker.py:267] model weights take 27.39GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 0.27GiB; the rest of the memory reserved for KV Cache is 7.85GiB.
INFO 02-15 03:29:12 executor_base.py:110] # CUDA blocks: 2572, # CPU blocks: 1310
INFO 02-15 03:29:12 executor_base.py:115] Maximum concurrency for 512 tokens per request: 80.38x
INFO 02-15 03:29:14 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_uti

Capturing CUDA graph shapes: 100%|██████████| 1/1 [00:00<00:00,  1.21it/s]

INFO 02-15 03:29:15 model_runner.py:1562] Graph capturing finished in 1 secs, took 0.05 GiB
INFO 02-15 03:29:15 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 5.29 seconds


In [8]:
from transformers import AutoTokenizer

def run_inference(prompt):
  tokenizer = AutoTokenizer.from_pretrained("Jlonge4/phi4-r1-merge")

  SYSTEM_PROMPT = """
  Respond in the following format:
  <reasoning>
  ...
  </reasoning>
  <answer>
  ...
  </answer>
  """

  SYSTEM = 'Your job is to evaluate whether a machine learning model has hallucinated or not. \n' + SYSTEM_PROMPT

  # Define prompts
  text = tokenizer.apply_chat_template([
      {"role" : "system", "content" : SYSTEM_PROMPT},
      {"role" : "user", "content" : prompt},
    ], tokenize = False, add_generation_prompt = True)

  prompts = [
      text
  ]

  # Generate responses
  outputs = llm.generate(prompts, sampling_params)

  # Print results
  for output in outputs:
      prompt = output.prompt
      generated_text = output.outputs[0].text
      print(f"Prompt: {prompt}")
      print('------------------'*40)
      print(f"Generated text: {generated_text}\n")

In [9]:
run_inference(prompt)

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.93s/it, est. speed input: 29.02 toks/s, output: 40.43 toks/s]

Prompt: <|im_start|>system<|im_sep|>
  Respond in the following format:
  <reasoning>
  ...
  </reasoning>
  <answer>
  ...
  </answer>
  <|im_end|><|im_start|>user<|im_sep|><|im_start|>system<|im_sep|>
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
<|im_end|><|im_start|>user<|im_sep|>Your job is to evaluate whether a machine learning model has hallucinated or not.
  A hallucination occurs when the response is coherent but factually incorrect or nonsensical
  outputs that are not grounded in the provided context.
  You are given the following information:
      ####INFO####
      [Knowledge]: The capital of France is Paris. The population is around 2 million.
      [User Input]: What is the population of the capital of France?
      [Model Response]: London, the capital of France has about 429 million people.
      ####END INFO####
      Based on the information provided is the model output a hallucination?
      <|im_end|><|im_start|>assistant<|im

In [10]:
reference = 'The capital of France is Paris. The population is around 2 million.'
query = 'What is the population of the capital of France?'
response = 'Paris, has about 2 million residents'

prompt = format_func(reference, query, response)

In [11]:
run_inference(prompt)

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.85s/it, est. speed input: 28.02 toks/s, output: 40.32 toks/s]

Prompt: <|im_start|>system<|im_sep|>
  Respond in the following format:
  <reasoning>
  ...
  </reasoning>
  <answer>
  ...
  </answer>
  <|im_end|><|im_start|>user<|im_sep|>Your job is to evaluate whether a machine learning model has hallucinated or not.
  A hallucination occurs when the response is coherent but factually incorrect or nonsensical
  outputs that are not grounded in the provided context.
  You are given the following information:
      ####INFO####
      [Knowledge]: The capital of France is Paris. The population is around 2 million.
      [User Input]: What is the population of the capital of France?
      [Model Response]: Paris, has about 2 million residents
      ####END INFO####
      Based on the information provided is the model output a hallucination?
      <|im_end|><|im_start|>assistant<|im_sep|>
----------------------------------------------------------------------------------------------------------------------------------------------------------------------